In [49]:
pip install pyspark>=3.5.1


Note: you may need to restart the kernel to use updated packages.


In [51]:
import pyspark
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import regexp_replace, col
from pyspark.sql import functions as F
from pyspark.sql.functions import col
from pyspark.sql.functions import col
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col, when
import numpy as np
from scipy.sparse import csr_matrix
from pyspark.sql.functions import col, monotonically_increasing_id, when
from pyspark.sql.types import FloatType
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark import SparkContext
import os
import time
from pyspark.sql import Window
from pyspark.sql.functions import avg
from pyspark.sql.functions import stddev






In [52]:



spark = SparkSession.builder.master("local[*]").appName("MyApp").getOrCreate()
os.environ["JAVA_HOME"] = r"C:\Users\pricc\OneDrive\Desktop\pyspark\app.py\Java\jdk-11"


In [53]:


#Does JAVA path exist
os.environ["JAVA_HOME"] = r"C:\Users\pricc\OneDrive\Desktop\pyspark\app.py\Java\jdk-11"
print("JAVA_HOME exists:", os.path.exists(os.environ["JAVA_HOME"]))


JAVA_HOME exists: True


In [54]:
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import regexp_replace, col
from pyspark.sql import functions as F
from pyspark.sql.functions import col
from pyspark.sql.functions import col
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col, when
import numpy as np
from scipy.sparse import csr_matrix
from pyspark.sql.functions import col, monotonically_increasing_id, when
from pyspark.sql.types import FloatType
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Imputer
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, when, monotonically_increasing_id
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.mllib.linalg.distributed import RowMatrix
from pyspark.sql.functions import collect_list, struct
from pyspark.sql.functions import avg, lit
from pyspark.sql.functions import col, when, monotonically_increasing_id, expr
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer



In [55]:
from pyspark.sql import SparkSession

# 1) Create SparkSession (only once per notebook)
spark = SparkSession.builder \
    .appName("JesterDataLoader") \
    .master("local[*]") \
    .getOrCreate()


In [56]:
joke_df = spark.read.format('csv').load(r"C:\Users\pricc\OneDrive\Desktop\jester-data-1.csv",  header=False,  inferSchema=True,  sep=",")

In [57]:
print(joke_df)

DataFrame[_c0: int, _c1: double, _c2: double, _c3: double, _c4: double, _c5: double, _c6: double, _c7: double, _c8: double, _c9: double, _c10: double, _c11: double, _c12: double, _c13: double, _c14: double, _c15: double, _c16: double, _c17: double, _c18: double, _c19: double, _c20: double, _c21: double, _c22: double, _c23: double, _c24: double, _c25: double, _c26: double, _c27: double, _c28: double, _c29: double, _c30: double, _c31: double, _c32: double, _c33: double, _c34: double, _c35: double, _c36: double, _c37: double, _c38: double, _c39: double, _c40: double, _c41: double, _c42: double, _c43: double, _c44: double, _c45: double, _c46: double, _c47: double, _c48: double, _c49: double, _c50: double, _c51: double, _c52: double, _c53: double, _c54: double, _c55: double, _c56: double, _c57: double, _c58: double, _c59: double, _c60: double, _c61: double, _c62: double, _c63: double, _c64: double, _c65: double, _c66: double, _c67: double, _c68: double, _c69: double, _c70: double, _c71: dou

In [58]:


# Step 1: Rename columns
joke_cols = ['ratingCount'] + [f'joke_{i}' for i in range(1, 101)]
joke_df = joke_df.toDF(*joke_cols)

# Step 2: Drop the 'ratingCount' column
joke_df = joke_df.drop('ratingCount')

# Step 3: Replace 99 with null (None)
for c in joke_df.columns:
    joke_df = joke_df.withColumn(c, when(col(c) == 99, None).otherwise(col(c)))

# Step 4: Limit to first 5000 users
joke_df = joke_df.limit(5000)

# Optional: Add userId column (if needed for indexing)
joke_df = joke_df.withColumn("userId", monotonically_increasing_id())

# --- If you want a NumPy or sparse matrix ---
# WARNING: This part requires collecting to driver. Not scalable beyond small data.
ratings_np = np.array(joke_df.drop("userId").collect())
ratings_matrix = np.array([[float(val) if val is not None else np.nan for val in row] for row in ratings_np])

from scipy.sparse import csr_matrix
ratings_sparse = csr_matrix(np.nan_to_num(ratings_matrix, nan=0.0))

In [96]:
print(ratings_matrix)

[[-7.82  8.79 -9.66 ...   nan   nan   nan]
 [ 4.08 -0.29  6.36 ...  0.34 -4.32  1.07]
 [  nan   nan   nan ...   nan   nan   nan]
 ...
 [-0.68 -2.48 -3.4  ...   nan   nan   nan]
 [ 1.02 -3.16  3.16 ... -0.68 -6.6  -1.75]
 [ 3.54  2.82 -2.14 ...  1.31  0.87  5.29]]


In [40]:

# 1) Identify rating columns (jokes)
rating_cols = [c for c in joke_df.columns if c.startswith("joke_")]

# 2) Replace 99 with 0 (or missing)
df = joke_df.limit(5000)  # limit rows if needed

for c in rating_cols:
    df = df.withColumn(c, when(col(c) == 99, 0.0).otherwise(col(c).cast("double")))

# 3) Add unique user ID column (rawUser)
df = df.withColumn("rawUser", monotonically_increasing_id())

# 4) Convert wide → long format: (rawUser, rawJoke, rating)
n = len(rating_cols)
stack_expr = ", ".join([f"'{c}', {c}" for c in rating_cols])
long_df = df.selectExpr("rawUser", f"stack({n}, {stack_expr}) as (rawJoke, rating)")

# 5) Convert joke string 'joke_42' → integer 42
long_df = long_df.withColumn("rawJoke", expr("int(substring(rawJoke, 6))"))

# 6) Index users and jokes to consecutive integers for ALS
user_indexer = StringIndexer(inputCol="rawUser", outputCol="userId").fit(long_df)
joke_indexer = StringIndexer(inputCol="rawJoke", outputCol="jokeId").fit(long_df)

indexed = user_indexer.transform(long_df)
indexed = joke_indexer.transform(indexed)

# 7) Select and cast final columns, drop nulls
data = indexed.select(
    col("userId").cast(IntegerType()),
    col("jokeId").cast(IntegerType()),
    col("rating").cast("double")
).na.drop()

# 8) Remove rows where userId == 0
data = data.filter(col("userId") != 0)

# Show first 5 rows of filtered data
data.show(5)

+------+------+------+
|userId|jokeId|rating|
+------+------+------+
|     1|     0|  4.08|
|     1|    12| -0.29|
|     1|    23|  6.36|
|     1|    34|  4.37|
|     1|    45| -2.38|
+------+------+------+
only showing top 5 rows



RMSE ALS TEST ON 5000 ratings

I am asking Spark to randomly split the rows of the data DataFrame into two parts, 80% of the rows go into train and 20% go into test.
This is done row-wise, not by user or item, meaning each (userId, jokeId, rating) row is randomly assigned to one of the two datasets according to the 80/20 ratio.

Build an ALS model 

In [59]:
#split data
train, test = data.randomSplit([0.8, 0.2], seed=42)

# Initialize ALS model
als = ALS(
    maxIter=10,
    regParam=0.1,
    userCol="userId",
    itemCol="jokeId",
    ratingCol="rating",
    nonnegative=True,
    coldStartStrategy="drop"  # drop NaN predictions during evaluation
)

#  Train ALS model
model = als.fit(train)

# Generate predictions on test set
predictions = model.transform(test)

#  Evaluate predictions with RMSE, MSE, MAE
evaluator_rmse = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
evaluator_mse = RegressionEvaluator(metricName="mse", labelCol="rating", predictionCol="prediction")
evaluator_mae = RegressionEvaluator(metricName="mae", labelCol="rating", predictionCol="prediction")

rmse_als = evaluator_rmse.evaluate(predictions)
mse_als = evaluator_mse.evaluate(predictions)
mae_als = evaluator_mae.evaluate(predictions)

print(f"ALS Model Performance :")
print(f"  RMSE_ALS: {rmse_als:.4f}")
print(f"  MSE_ALS:  {mse_als:.4f}")
print(f"  MAE_ALS:  {mae_als:.4f}")

ALS Model Performance :
  RMSE_ALS: 4.5465
  MSE_ALS:  20.6707
  MAE_ALS:  3.5603


In [60]:
start_model = time.time()

als = ALS(
    maxIter=10,
    regParam=0.1,
    userCol="userId",
    itemCol="jokeId",
    ratingCol="rating",
    nonnegative=True,
    coldStartStrategy="drop"
)

model = als.fit(train)

end_model = time.time()

# Measure prediction time
start_pred = time.time()
predictions = model.transform(test)
end_pred = time.time()

# Evaluate
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)

# Output results
print(f"ALS model performance:")
print(f"  RMSE: {rmse:.4f}")
print(f"  Model time:      {end_model - start_model:.3f} sec")
print(f"  Prediction time: {end_pred - start_pred:.3f} sec")
print(f"ALS model time / prediction time:\n 1  [{end_model - start_model:.3f} sec / {end_pred - start_pred:.3f} sec]")

ALS model performance:
  RMSE: 4.5465
  Model time:      4.498 sec
  Prediction time: 0.102 sec
ALS model time / prediction time:
 1  [4.498 sec / 0.102 sec]


BASELINE GLOBAL MEAN USING PYSPARK

In [61]:
# Compute global average rating from training data
mean_rating = train.select(avg("rating")).first()[0]

# Add that as a constant "prediction" column to test set
baseline_preds = test.withColumn("prediction", lit(mean_rating))

# Evaluate RMSE
baseline_rmse = RegressionEvaluator(
    labelCol="rating",
    predictionCol="prediction",
    metricName="rmse"
).evaluate(baseline_preds)

print(f"Baseline RMSE (global mean): {baseline_rmse:.4f}")

Baseline RMSE (global mean): 5.1901


 Mean-Centering Normalization

In [63]:

# Compute user mean
user_avg = train.groupBy("userId").agg(avg("rating").alias("mean_rating"))

# Join and mean-center ratings
train_centered = train.join(user_avg, on="userId")
train_centered = train_centered.withColumn("rating_centered", col("rating") - col("mean_rating"))

# Train ALS on mean-centered data
als_centered = ALS(
    maxIter=10,
    regParam=0.1,
    userCol="userId",
    itemCol="jokeId",
    ratingCol="rating_centered",
    nonnegative=True,
    coldStartStrategy="drop"
).fit(train_centered)

# Predict
test_centered = test.join(user_avg, on="userId")
preds_centered = als_centered.transform(test_centered)
preds_centered = preds_centered.withColumn("prediction", col("prediction") + col("mean_rating"))



 Z-Score Normalization

In [64]:


# Compute user mean and stddev
user_stats = train.groupBy("userId").agg(
    avg("rating").alias("mean_rating"),
    stddev("rating").alias("std_rating")
)

# Join and z-score normalize
train_z = train.join(user_stats, on="userId")
train_z = train_z.withColumn("rating_z", (col("rating") - col("mean_rating")) / col("std_rating"))

# Train ALS on z-score data
als_z = ALS(
    maxIter=10,
    regParam=0.1,
    userCol="userId",
    itemCol="jokeId",
    ratingCol="rating_z",
    nonnegative=True,
    coldStartStrategy="drop"
).fit(train_z)

# Predict and unnormalize
test_z = test.join(user_stats, on="userId")
preds_z = als_z.transform(test_z)
preds_z = preds_z.withColumn("prediction", (col("prediction") * col("std_rating")) + col("mean_rating"))

In [65]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

print("No Normalization:", evaluator.evaluate(predictions))
print("Mean Centered:", evaluator.evaluate(preds_centered))
print("Z-Score:", evaluator.evaluate(preds_z))

No Normalization: 4.546503340691266
Mean Centered: 4.436330404001126
Z-Score: 4.435515030094131


In [66]:
def normalize_data(data, method=None):
    """
    Normalize ratings using mean-centering or Z-score normalization.

    Parameters:
    - data: Spark DataFrame with columns ['userId', 'jokeId', 'rating']
    - method: None, 'center', or 'zscore'

    Returns:
    - normalized Spark DataFrame
    """
    if method is None:
        return data

    stats = data.groupBy("userId").agg(
        avg("rating").alias("user_mean"),
        stddev("rating").alias("user_std")
    )

    joined = data.join(stats, on="userId")

    if method == 'center':
        normed = joined.withColumn("rating", col("rating") - col("user_mean"))
    elif method == 'zscore':
        normed = joined.withColumn("rating", (col("rating") - col("user_mean")) / col("user_std"))
    else:
        raise ValueError("Normalization must be 'center', 'zscore', or None")

    return normed.select("userId", "jokeId", "rating").na.drop()

def run_als_with_normalization(data, norm_method=None, maxIter=10, regParam=0.1):
    """
    Train ALS with optional normalization and evaluate performance.

    Parameters:
    - data: Spark DataFrame with ['userId', 'jokeId', 'rating']
    - norm_method: None, 'center', or 'zscore'
    """
    normalized_data = normalize_data(data, method=norm_method)
    train, test = normalized_data.randomSplit([0.8, 0.2], seed=42)

    start_model = time.time()

    als = ALS(
        maxIter=maxIter,
        regParam=regParam,
        userCol="userId",
        itemCol="jokeId",
        ratingCol="rating",
        nonnegative=True,
        coldStartStrategy="drop"
    )
    model = als.fit(train)
    end_model = time.time()

    start_pred = time.time()
    predictions = model.transform(test)
    end_pred = time.time()

    evaluator_rmse = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
    evaluator_mse  = RegressionEvaluator(metricName="mse",  labelCol="rating", predictionCol="prediction")
    evaluator_mae  = RegressionEvaluator(metricName="mae",  labelCol="rating", predictionCol="prediction")

    rmse = evaluator_rmse.evaluate(predictions)
    mse  = evaluator_mse.evaluate(predictions)
    mae  = evaluator_mae.evaluate(predictions)

    print(f"\nALS Model Performance ({norm_method or 'no normalization'}):")
    print(f"  RMSE_ALS: {rmse:.4f}")
    print(f"  MSE_ALS:  {mse:.4f}")
    print(f"  MAE_ALS:  {mae:.4f}")
    print(f"ALS model time / prediction time:\n 1  [{end_model - start_model:.3f} sec / {end_pred - start_pred:.3f} sec]")

# Run all three variants
run_als_with_normalization(data, norm_method=None)
run_als_with_normalization(data, norm_method="center")
run_als_with_normalization(data, norm_method="zscore")


ALS Model Performance (no normalization):
  RMSE_ALS: 4.5465
  MSE_ALS:  20.6707
  MAE_ALS:  3.5603
ALS model time / prediction time:
 1  [4.717 sec / 0.070 sec]

ALS Model Performance (center):
  RMSE_ALS: 4.3924
  MSE_ALS:  19.2935
  MAE_ALS:  3.3894
ALS model time / prediction time:
 1  [6.122 sec / 0.142 sec]

ALS Model Performance (zscore):
  RMSE_ALS: 0.9572
  MSE_ALS:  0.9162
  MAE_ALS:  0.7768
ALS model time / prediction time:
 1  [6.472 sec / 0.159 sec]


Matrix factorization-based collaborative filtering model using ALS 

In [30]:
# 2. Prepare data (wide → long)
rating_cols = [c for c in joke_df.columns if c.startswith("joke_")]
df = joke_df.limit(5000)  # limit for speed

# Replace 99s with 0.0 (or drop if you'd rather exclude them)
for c in rating_cols:
    df = df.withColumn(c, when(col(c) == 99, 0.0).otherwise(col(c).cast("double")))

df = df.withColumn("rawUser", monotonically_increasing_id())

# Wide to long format
n = len(rating_cols)
stack_expr = ", ".join([f"'{c}', {c}" for c in rating_cols])
long_df = df.selectExpr("rawUser", f"stack({n}, {stack_expr}) as (rawJoke, rating)")

# Extract numeric joke ID
long_df = long_df.withColumn("rawJoke", expr("int(substring(rawJoke, 6))"))

# Index users and jokes
user_indexer = StringIndexer(inputCol="rawUser", outputCol="userId").fit(long_df)
joke_indexer = StringIndexer(inputCol="rawJoke", outputCol="jokeId").fit(long_df)

indexed = user_indexer.transform(long_df)
indexed = joke_indexer.transform(indexed)

# Cast to int/double
data = indexed.select(
    col("userId").cast(IntegerType()),
    col("jokeId").cast(IntegerType()),
    col("rating").cast("double")
).na.drop()

# 3. Split into train/test
train, test = data.randomSplit([0.8, 0.2], seed=42)

# 4. Train ALS model (collaborative filtering)
als = ALS(
    userCol="userId",
    itemCol="jokeId",
    ratingCol="rating",
    maxIter=10,
    regParam=0.1,
    nonnegative=True,
    coldStartStrategy="drop"
)

model = als.fit(train)

# 5. Predict and evaluate
predictions = model.transform(test)

evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="rating",
    predictionCol="prediction"
)

rmse_cbf = evaluator.evaluate(predictions)
evaluator_mse = RegressionEvaluator(metricName="mse", labelCol="rating", predictionCol="prediction")
mse_cbf = evaluator_mse.evaluate(predictions)
evaluator_mae = RegressionEvaluator(metricName="mae", labelCol="rating", predictionCol="prediction")
mae_cbf = evaluator_mae.evaluate(predictions)
print(f" Collaborative filtering model using ALS ")
print(f"  RMSE: {rmse_cbf:.4f}")
print(f"  MSE:  {mse_cbf:.4f}")
print(f"  MAE:  {mae_cbf:.4f}")

 Collaborative filtering model using ALS 
  RMSE: 4.5524
  MSE:  20.7245
  MAE:  3.5627


In [48]:
print(type(joke_df))

<class 'pyspark.sql.dataframe.DataFrame'>


In [ ]:
print(f"ALS Model Performance :")
print(f"  RMSE_ALS: {rmse_als:.4f}")
print(f"  MSE_ALS:  {mse_als:.4f}")
print(f"  MAE_ALS:  {mae_als:.4f}")
print(f"ALS model performance:")
print(f"  RMSE: {rmse:.4f}")
print(f"  Model time:      {end_model - start_model:.3f} sec")
print(f"  Prediction time: {end_pred - start_pred:.3f} sec")
print(f"ALS model time / prediction time:\n 1  [{end_model - start_model:.3f} sec / {end_pred - start_pred:.3f} sec]")


ALS Model Performance :
  RMSE_ALS: 4.5465
  MSE_ALS:  20.6707
  MAE_ALS:  3.5603
ALS model performance:
  RMSE: 4.5465
  Model time:      4.121 sec
  Prediction time: 0.080 sec
ALS model time / prediction time:
 1  [4.121 sec / 0.080 sec]


CONCLUSION

The UBCF and SVD model in my project 4 performed better than the ALS as seen below.

The ALS model metrics below are higher than the prior metrics from project 4. The ALS model time and prediction time is also higher.
These metrics quantify how well a recommender system predicts user ratings, compared to actual ratings.

ALS RMSE emphasizes large errors, so the Lower it is the better. ALS with no normalization uses raw ratings. Errors are relatively high at 4.565. Compared to the UBCF and SVD model in my project 4 with RMSE of 3.840 for UBCF and 3.2 for SVD. The Prediction time for ubcf was 0.090sec/0.128sec.  ALS assumes that all users rate on the same scale, which is rarely true, some users might always rate jokes lower/higher. The prediction time of the ALS models is very high ranging from 4.717 sec to 6.472 sec. See below other ALS models metrics and time.



ALS Model Performance (no normalization):
  RMSE_ALS: 4.5465,
  MSE_ALS:  20.6707,
  MAE_ALS:  3.5603,
ALS model time / prediction time:
 1  [4.717 sec / 0.070 sec],

With Normalizing which is done by subtracting each user’s average rating.  We see a Slight improvement over raw data, The model now understands user bias better (e.g., User A may always rate jokes 2 points higher than User B). But the prediction time increases sue to preprocessing
ALS Model Performance (center):
  RMSE_ALS: 4.3924,
  MSE_ALS:  19.2935,
  MAE_ALS:  3.3894,
ALS model time / prediction time:
 1  [6.122 sec / 0.142 sec]

Z-score captures both bias and variability of each user’s rating scale. This puts all users on a more comparable footing for ALS training. This model is offers best accuracy by far with slightly higher compute cost.

ALS Model Performance (zscore):
  RMSE_ALS: 0.9572,
  MSE_ALS:  0.9162,
  MAE_ALS:  0.7768,
ALS model time / prediction time:
 1  [6.472 sec / 0.159 sec]
--------------------------------Project 4 metrics on Jester data --------------------
Model   RMSE     MSE    MAE
0  UBCF2  3.880  15.058  3.061,
1   SVD2  3.206  10.279  2.418,
2  IBCF2  4.251  18.069  3.395,

MODEL TIME AND PREDICTION TIME COMPARISON and 
SVD run fold/sample [model time/prediction time] 1 [0.020sec/0.001sec],
UBCF run fold/sample [model time/prediction time] 1 [0.090sec/0.128sec],
IBCF run fold/sample [model time/prediction time] 1 [0.003sec/0.005sec],

REGULARIZATION
RegParam was applied automatically to both user and item latent factors.
Spark’s ALS uses a conjugate gradient solver and optimizes in a distributed fashion.

Spark does not expose fine-grained control like separate regularization for users/items or custom loss functions.  This leads to Optimization for large-scale distributed training, built-in regularization is efficient and easy to tune via regParam.
Non-Spark Regularization often gives more control with reg_all, reg_bu, reg_bi for bias terms and separate regularization for user and item embeddings. This comes in handy for Easier experimentation, customization and is better suited for smaller datasets.

It does not offer distributed scalability like Spark.

With a PySpark/MLlib workflow your model lives in the JVM’s distributed memory (across executors), 
so after that one expensive fit() call you can call .transform() on new data very cheaply without retraining or pulling 
everything back to the driver. In contrast, a pure pandas-based pipeline typically retrains or recomputes locally for each batch
(or requires you to keep everything in a global in-memory object),
Pandas also don't automatically distribute work across a cluster.

For a trained ALS (or any Spark ML) model, it stays “warm” in memory and can serve many prediction
requests at scale, making pySpark a better choice for production deployment on large datasets or high-throughput scenarios.
In conclusion, moving to a distributed architecture would seem advisable when data sets are large,
processing is computationally demanding and / or minimizing processing time is critical.

The process of setting up the enviroment and the PY, java and Pyspark versions that work together was such a headache. That put me against this model too.
